In [1]:
import pandas as pd
import plotly.express as px
import numpy as np

In [2]:
f = '../data/ClimaLab_2023-10-27_2025-04-30.parquet'
tmx = pd.read_parquet(f)
tmx = tmx.reset_index().rename(columns={"index": "date"})
tmx

variable,date,dhi,dni,ghi,p_atm,rain_acc,rh,solar_altitude,tdb,uv,wd,ws
0,2023-10-27 16:40:00,NaN,443.2,238.0,873.2120,17.1,61.14,18.644204,28.07,0.018,226.50,1.569
1,2023-10-27 16:50:00,NaN,NaN,187.5,873.2559,6.3,62.76,16.430841,27.78,0.014,171.60,1.631
2,2023-10-27 17:00:00,1360.00,NaN,167.5,873.2161,27.8,64.26,14.204506,28.00,0.011,314.70,0.479
3,2023-10-27 17:10:00,74.14,520.5,163.1,873.1834,5.4,66.82,11.967374,28.69,0.009,163.60,0.504
4,2023-10-27 17:20:00,67.62,269.2,126.1,873.2775,1.5,60.92,9.722172,28.77,0.006,260.30,0.964
...,...,...,...,...,...,...,...,...,...,...,...,...
79031,2025-04-29 23:20:00,NaN,NaN,NaN,861.8378,0.0,43.17,-52.087453,26.46,0.000,164.30,0.808
79032,2025-04-29 23:30:00,NaN,NaN,NaN,861.8724,0.0,46.12,-53.192891,26.30,0.000,337.00,0.518
79033,2025-04-29 23:40:00,NaN,NaN,NaN,861.8623,0.0,46.82,-54.162933,25.57,0.000,70.61,1.358
79034,2025-04-29 23:50:00,NaN,NaN,NaN,861.9176,0.0,47.47,-54.986312,25.59,0.000,133.60,0.625


In [3]:
tidy = tmx.melt(id_vars="date", var_name="variable", value_name="valor")
tidy

,date,variable,valor
0,2023-10-27 16:40:00,dhi,NaN
1,2023-10-27 16:50:00,dhi,NaN
2,2023-10-27 17:00:00,dhi,1360.000
3,2023-10-27 17:10:00,dhi,74.140
4,2023-10-27 17:20:00,dhi,67.620
...,...,...,...
869391,2025-04-29 23:20:00,ws,0.808
869392,2025-04-29 23:30:00,ws,0.518
869393,2025-04-29 23:40:00,ws,1.358
869394,2025-04-29 23:50:00,ws,0.625


In [ ]:

# Asegura 'date' como columna

fig = px.line(
    tmx,
    x="date",
    y=["ghi", "dni", "dhi"],
    title="Irradiancias (interactivo)",
    labels={"value": "W/m²", "variable": "Magnitud", "date": "Tiempo"}
)
fig.update_xaxes(rangeslider_visible=True)
fig.show()


In [ ]:


fig = px.line(
    tidy,
    x="date", y="valor",
    facet_col="variable", facet_col_wrap=3,
    height=700,
    title="Panel de series por variable",
    labels={"valor": "Valor", "date": "Tiempo"}
)
# No forzar el mismo eje-y (unidades distintas)
fig.for_each_yaxis(lambda a: a.update(matches=None))
fig.update_xaxes(showspikes=True)
fig.show()


In [ ]:

tmx["hora"] = tmx["date"].dt.hour
tmx["mes"]  = tmx["date"].dt.month

hm = (tmx.groupby(["hora", "mes"])["tdb"]
        .mean()
        .unstack("mes")
        .reindex(index=range(24), columns=range(1, 13)))

fig = px.imshow(
    hm,
    origin="lower",
    aspect="auto",
    color_continuous_scale="jet",
    labels=dict(x="Mes", y="Hora del día", color="tdb [°C]"),
)

fig.show()
